## JSSP Example

This notebook demonstraits how to use prallel tabu seach to generate a an initial population for a genetic algorithm.

In [1]:
from JSSP.data import Data
import os
"""
initialize static data from csv files
"""
data_directory = '../data/data_set2'
Data.initialize_data(data_directory + '/sequenceDependencyMatrix.csv',
                     data_directory + '/machineRunSpeed.csv',
                     data_directory + '/jobTasks.csv')

# uncomment this to print the data that was read in
# Data.print_data()

In [2]:
# global variables
runtimes = {
    'tabu search': 300,
    'genetic algorithm': 300
}

output_dir = './jssp_output'

### Tabu Search

In [3]:
from JSSP.tabu_search import TabuSearchManager

# TS parameters
ts_runtime = runtimes['tabu search']
num_processes = 50
tabu_list_size = 50
neighborhood_size = 300
neighborhood_wait = 0.15
probability_change_machine = 0.8

ts_manager = TabuSearchManager(ts_runtime, 
                               num_processes, 
                               tabu_list_size, 
                               neighborhood_size, 
                               neighborhood_wait, 
                               probability_change_machine)

ts_manager.start()

print("Best solution found by Tabu Search:")
ts_manager.best_solution.pprint()

Best solution found by Tabu Search:
makespan = 4057.0162601626016
machine_makespans = [4057.0162601626016, 4043.3620689655177, 4031.0789473684213, 4052.3975903614464, 4028.5495495495497, 4036.9399999999996, 4037.428571428571, 4033.8874172185433]
operation_list =
[[   14     0     0     0 14120]
 [    0     0     0     5 19116]
 [   35     0     0     5  6339]
 ...
 [   31     6     5     2 23583]
 [    6     4     2     0  7660]
 [   44     5     4     7 10137]]


### Genetic Algorithm 

In [4]:
from JSSP.genetic_algorithm import search
from JSSP.solution import generate_feasible_solution

# GA parameters
ga_runtime = runtimes['genetic algorithm']
population_size = 100
mutation_probability=0.05

# create initial population using solutions found by tabu search
initial_population = ts_manager.all_solutions

# generate random solutions for remaining population
for _ in range(abs(population_size - len(initial_population))):
    initial_population.append(generate_feasible_solution())
    
result = search(ga_runtime, initial_population, mutation_probability, verbose=True)

print("Best solution found by Tabu Search:")
result.pprint()

# result.create_schedule(output_dir)

total iterations = 1713523
total number of infeasible solutions generated = 33944
Best solution found by Tabu Search:
makespan = 4057.0162601626016
machine_makespans = [4057.0162601626016, 4043.3620689655177, 4031.0789473684213, 4052.3975903614464, 4028.5495495495497, 4036.9399999999996, 4037.428571428571, 4033.8874172185433]
operation_list =
[[   14     0     0     0 14120]
 [    0     0     0     5 19116]
 [   35     0     0     5  6339]
 ...
 [   31     6     5     2 23583]
 [    6     4     2     0  7660]
 [   44     5     4     7 10137]]
